In [1]:
import sys, os

import numpy as np

import config
import mode_choice
import mc_util
import scenario_editor as se

mc = mode_choice.Mode_Choice(config,run_now = False)

In [2]:
mc.load_input()
mc.run_model(all_purposes = True)

# default behavior: write mode share tables

✓ TAZ / land use / parking / zonal variables read. Time elapsed: 56.26 seconds
✓ skims read. Time elapsed: 74.04 seconds
✓ trip table read. Time elapsed: 82.35 seconds
✓ zonal variable tables generated. Time elapsed: 82.77 seconds
Running for all purposes...
Mode choice for HBW started.
✓ Parameter table for HBW generated. Time elapsed: 82.91 seconds
✓ Trips for 0_PK calculated. Time elapsed: 98.46 seconds
✓ Trips for 1_PK calculated. Time elapsed: 112.50 seconds
✓ Trips for 0_OP calculated. Time elapsed: 126.72 seconds
✓ Trips for 1_OP calculated. Time elapsed: 140.67 seconds
Mode choice for HBO started.
✓ Parameter table for HBO generated. Time elapsed: 141.52 seconds
✓ Trips for 0_PK calculated. Time elapsed: 151.19 seconds
✓ Trips for 1_PK calculated. Time elapsed: 161.69 seconds
✓ Trips for 0_OP calculated. Time elapsed: 172.11 seconds
✓ Trips for 1_OP calculated. Time elapsed: 182.55 seconds
Mode choice for NHB started.
✓ Parameter table for NHB generated. Time elapsed: 183.30 se

In [3]:
mc_util.write_summary_by_subregion(mc,out_path = config.out_path)

In [4]:
# inspect results
# mc_util.write_trip_tables(mc, config.out_path +  '2040_trips_no_scenario.omx')
mc_util.display_mode_share(mc)

,Bike,DA,DAT_B,DAT_CR,DAT_LB,DAT_RT,SR2,SR3+,WAT,Walk
0_PK,9.93%,19.30%,0.00%,1.12%,0.30%,0.60%,18.90%,0.61%,24.67%,24.56%
1_PK,2.19%,57.41%,0.04%,0.93%,0.34%,0.19%,27.68%,0.80%,3.80%,6.62%
0_OP,10.18%,16.95%,0.00%,0.15%,0.22%,0.09%,23.17%,0.19%,13.77%,35.27%
1_OP,1.15%,52.84%,0.00%,0.12%,0.44%,0.09%,35.00%,0.22%,1.69%,8.45%


In [ ]:

import imp
imp.reload(se)


In [4]:
from IPython.display import display
display(config.scenario_switches)

{'clean_vehicle': False,
 'growth_shift': False,
 'transit_improvements': False,
 'active_transportation_improvements': False,
 'TDM': False,
 'CAV': False,
 'smart_mobility': False,
 'congestion_charge': False}

# Implementing every scenario by its default setup.

In [13]:
# For loop to automatically implement each scenario.
for scenario in [
 #   'clean_vehicle', # init
#    'growth_shift', # load input
#    'transit_improvements', # load input
#    'active_transportation_improvements', # load input
#    'TDM', # requires a full run
#    'CAV', 
#    'smart_mobility', # cannot be implemented
    'congestion_charge']:
# step 1: revert to non-implementation
    mc = mode_choice.Mode_Choice(config, run_now=False)
    mc.load_input()
    # mc.run_model(all_purposes = True)
# step 2: turn on scenario, turn off everything else
    for key in config.scenario_switches:
        config.scenario_switches[key] = False
    config.scenario_switches[scenario] = True
    
    if scenario == 'TDM':
        mc.run_model(all_purposes = True)
    
    se.implement_scenarios(mc)
# step 3: save key outputs
    try:
        os.mkdir(config.out_path + scenario)
    except:
        pass
    mc_util.write_mode_share_to_excel(mc,purpose = 'all',out_excel_fn = config.out_path + scenario +'_mode_share.xlsx')
    
    for by in [None, 'peak','veh_own','purpose']:
        mc_util.vmt_by_neighborhood(mc, by = by, out_fn = config.out_path + scenario+f'_vmt_by_neighborhood_{by}.csv')
        mc_util.pmt_by_neighborhood(mc, by = by, out_fn = config.out_path + scenario+f'_pmt_by_neighborhood_{by}.csv')
        mc_util.mode_share_by_neighborhood(mc, by = by , out_fn = config.out_path + scenario+f'_mode_share_nhood_{by}.csv')
    

    mc_util.write_summary_by_subregion(mc,out_path = config.out_path + scenario + '/')
    
    mc_util.write_trip_tables(mc, r'..\output\2040_trips_'+ scenario + '.omx')
    
    

✓ TAZ / land use / parking / zonal variables read. Time elapsed: 0.11 seconds
✓ skims read. Time elapsed: 18.00 seconds
✓ trip table read. Time elapsed: 27.30 seconds
✓ zonal variable tables generated. Time elapsed: 27.85 seconds
"Congestion Charge" enabled: $5 toll charge for auto trips entering central Boston.
Running for all purposes...
Mode choice for HBW started.
✓ Parameter table for HBW generated. Time elapsed: 27.98 seconds
✓ Trips for 0_PK calculated. Time elapsed: 43.86 seconds
✓ Trips for 1_PK calculated. Time elapsed: 54.55 seconds
✓ Trips for 0_OP calculated. Time elapsed: 67.19 seconds
✓ Trips for 1_OP calculated. Time elapsed: 81.59 seconds
Mode choice for HBO started.
✓ Parameter table for HBO generated. Time elapsed: 82.77 seconds
✓ Trips for 0_PK calculated. Time elapsed: 90.44 seconds
✓ Trips for 1_PK calculated. Time elapsed: 97.99 seconds
✓ Trips for 0_OP calculated. Time elapsed: 105.53 seconds
✓ Trips for 1_OP calculated. Time elapsed: 113.27 seconds
Mode choice 

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


# Parameter sensitivity for selected scenarios

In [2]:
# Clean vehicle: change clean_veh_amount in scenario_editor
for clean_veh_amount in [0.05, 0.07,0.09,0.11,0.13, 0.15]:
    
    mc = mode_choice.Mode_Choice(config, run_now = False)
    mc.load_input()
    
    for key in config.scenario_switches:
        config.scenario_switches[key] = False
    config.scenario_switches['clean_vehicle'] = True
    
    se.clean_veh_amount = clean_veh_amount
    se.implement_scenarios(mc)
    
    dir_out = config.out_path + 'sensitivity/' + 'clean_veh/'
    try: os.makedirs(config.out_path + 'sensitivity/' + 'clean_veh/')
    except: pass

    mc_util.write_mode_share_to_excel(mc,purpose = 'all',out_excel_fn = dir_out + str(clean_veh_amount) +'_mode_share.xlsx')
    
    for by in [None, 'peak','veh_own','purpose']:
        mc_util.vmt_by_neighborhood(mc, by = by, out_fn = dir_out + str(clean_veh_amount)+f'_vmt_by_neighborhood_{by}.csv')
        mc_util.pmt_by_neighborhood(mc, by = by, out_fn = dir_out + str(clean_veh_amount)+f'_pmt_by_neighborhood_{by}.csv')
        mc_util.mode_share_by_neighborhood(mc, by = by , out_fn = dir_out + str(clean_veh_amount)+f'_mode_share_nhood_{by}.csv')
    
    os.makedirs(dir_out + str(clean_veh_amount))
    mc_util.write_summary_by_subregion(mc,out_path = dir_out + str(clean_veh_amount) + '/')
    
    # mc_util.write_trip_tables(mc, dir_out + str(clean_veh_amount) + '_2040_trips.omx')
        
    

✓ TAZ / land use / parking / zonal variables read. Time elapsed: 0.10 seconds
✓ skims read. Time elapsed: 18.05 seconds
✓ trip table read. Time elapsed: 26.31 seconds
✓ zonal variable tables generated. Time elapsed: 26.73 seconds
"Clean Vehicle" enabled: Reduce driving cost (by default $0.05 / mile)
Running for all purposes...
Mode choice for HBW started.
✓ Parameter table for HBW generated. Time elapsed: 26.81 seconds
✓ Trips for 0_PK calculated. Time elapsed: 41.91 seconds
✓ Trips for 1_PK calculated. Time elapsed: 55.44 seconds
✓ Trips for 0_OP calculated. Time elapsed: 69.18 seconds
✓ Trips for 1_OP calculated. Time elapsed: 82.85 seconds
Mode choice for HBO started.
✓ Parameter table for HBO generated. Time elapsed: 83.66 seconds
✓ Trips for 0_PK calculated. Time elapsed: 93.08 seconds
✓ Trips for 1_PK calculated. Time elapsed: 102.35 seconds
✓ Trips for 0_OP calculated. Time elapsed: 111.88 seconds
✓ Trips for 1_OP calculated. Time elapsed: 121.15 seconds
Mode choice for NHB star

C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future be

C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future be

C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future be

C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future be

...

C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future be

C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future be

C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future be

C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future be

C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\xy.chang\AppData\Local\conda\conda\envs\py36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future be

In [3]:
# Growth shift: change land_use_shift_factor in scenario_editor
for shift_factor in np.linspace(0.1, 1, 10):
    mc = mode_choice.Mode_Choice(config, run_now = False)
    mc.load_input()
    
    for key in config.scenario_switches:
        config.scenario_switches[key] = False
    config.scenario_switches['growth_shift'] = True
    
    se.land_use_shift_factor = shift_factor
    se.implement_scenarios(mc)
    
    dir_out = config.out_path + 'sensitivity/' + 'growth_shift/'
    try: os.makedirs(dir_out)
    except: pass

    # mc_util.write_mode_share_to_excel(mc,purpose = 'all',out_excel_fn = dir_out + str(shift_factor) +'_mode_share.xlsx')
    
    #for by in [None, 'peak','veh_own','purpose']:
    #    mc_util.vmt_by_neighborhood(mc, by = by, out_fn = dir_out + str(shift_factor) +f'_vmt_by_neighborhood_{by}.csv')
    #    mc_util.pmt_by_neighborhood(mc, by = by, out_fn = dir_out + str(shift_factor) +f'_pmt_by_neighborhood_{by}.csv')
    #    mc_util.mode_share_by_neighborhood(mc, by = by , out_fn = dir_out + str(shift_factor) +f'_mode_share_nhood_{by}.csv')
    
    os.makedirs(dir_out + str(shift_factor))
    mc_util.write_summary_by_subregion(mc,out_path = dir_out + str(shift_factor) + '/')
    
    # mc_util.write_trip_tables(mc, dir_out + str(shift_factor) + '_2040_trips.omx')
        
    

✓ TAZ / land use / parking / zonal variables read. Time elapsed: 0.10 seconds
✓ skims read. Time elapsed: 19.44 seconds
✓ trip table read. Time elapsed: 30.91 seconds
✓ zonal variable tables generated. Time elapsed: 31.36 seconds
"Land Use" enabled: Shift a certain amount of population growth to growth-intensive TAZs (by default 50%).
Running for all purposes...
Mode choice for HBW started.
✓ Parameter table for HBW generated. Time elapsed: 69.30 seconds
✓ Trips for 0_PK calculated. Time elapsed: 83.21 seconds
✓ Trips for 1_PK calculated. Time elapsed: 97.08 seconds
✓ Trips for 0_OP calculated. Time elapsed: 111.21 seconds
✓ Trips for 1_OP calculated. Time elapsed: 125.13 seconds
Mode choice for HBO started.
✓ Parameter table for HBO generated. Time elapsed: 125.86 seconds
✓ Trips for 0_PK calculated. Time elapsed: 135.36 seconds
✓ Trips for 1_PK calculated. Time elapsed: 145.10 seconds
✓ Trips for 0_OP calculated. Time elapsed: 154.84 seconds
✓ Trips for 1_OP calculated. Time elapsed:

✓ Trips for 1_OP calculated. Time elapsed: 161.78 seconds
Mode choice for NHB started.
✓ Parameter table for NHB generated. Time elapsed: 162.44 seconds
✓ Trips for 0_PK calculated. Time elapsed: 173.10 seconds
✓ Trips for 1_PK calculated. Time elapsed: 186.66 seconds
✓ Trips for 0_OP calculated. Time elapsed: 198.04 seconds
✓ Trips for 1_OP calculated. Time elapsed: 209.16 seconds
Mode choice for HBSc1 started.
✓ Parameter table for HBSc1 generated. Time elapsed: 209.90 seconds
✓ Trips for 0_PK calculated. Time elapsed: 217.71 seconds
✓ Trips for 1_PK calculated. Time elapsed: 225.32 seconds
✓ Trips for 0_OP calculated. Time elapsed: 232.93 seconds
✓ Trips for 1_OP calculated. Time elapsed: 240.70 seconds
Mode choice for HBSc2 started.
✓ Parameter table for HBSc2 generated. Time elapsed: 241.26 seconds
✓ Trips for 0_PK calculated. Time elapsed: 248.76 seconds
✓ Trips for 1_PK calculated. Time elapsed: 258.89 seconds
✓ Trips for 0_OP calculated. Time elapsed: 266.98 seconds
✓ Trips for

✓ Trips for 0_OP calculated. Time elapsed: 270.09 seconds
✓ Trips for 1_OP calculated. Time elapsed: 278.24 seconds
Mode choice for HBSc3 started.
✓ Parameter table for HBSc3 generated. Time elapsed: 278.92 seconds
✓ Trips for 0_PK calculated. Time elapsed: 288.01 seconds
✓ Trips for 1_PK calculated. Time elapsed: 297.36 seconds
✓ Trips for 0_OP calculated. Time elapsed: 306.63 seconds
✓ Trips for 1_OP calculated. Time elapsed: 316.00 seconds
Scenario run is finished. You may now call methods in mc_util to produce output summaries.
✓ TAZ / land use / parking / zonal variables read. Time elapsed: 0.08 seconds
✓ skims read. Time elapsed: 19.78 seconds
✓ trip table read. Time elapsed: 28.51 seconds
✓ zonal variable tables generated. Time elapsed: 30.08 seconds
"Land Use" enabled: Shift a certain amount of population growth to growth-intensive TAZs (by default 50%).
Running for all purposes...
Mode choice for HBW started.
✓ Parameter table for HBW generated. Time elapsed: 68.04 seconds
✓ T

In [4]:
# Congestion charge: change congestion_charge_fee in scenario_editor
for cc_fee in np.linspace(5, 50, 10):
    mc = mode_choice.Mode_Choice(config, run_now = False)
    mc.load_input() 
    
    for key in config.scenario_switches:
        config.scenario_switches[key] = False
    config.scenario_switches['congestion_charge'] = True
    
    se.congestion_charge_fee = cc_fee
    se.implement_scenarios(mc)
    
    dir_out = config.out_path + 'sensitivity/' + 'congestion_charge/'
    try: os.makedirs(dir_out)
    except: pass

    #mc_util.write_mode_share_to_excel(mc,purpose = 'all',out_excel_fn = dir_out + str(cc_fee) +'_mode_share.xlsx')
    
    #for by in [None, 'peak','veh_own','purpose']:
    #    mc_util.vmt_by_neighborhood(mc, by = by, out_fn = dir_out + str(cc_fee) +f'_vmt_by_neighborhood_{by}.csv')
    #    mc_util.pmt_by_neighborhood(mc, by = by, out_fn = dir_out + str(cc_fee) +f'_pmt_by_neighborhood_{by}.csv')
    #    mc_util.mode_share_by_neighborhood(mc, by = by , out_fn = dir_out + str(cc_fee) +f'_mode_share_nhood_{by}.csv')
    
    os.makedirs(dir_out + str(cc_fee))
    mc_util.write_summary_by_subregion(mc,out_path = dir_out + str(cc_fee) + '/')
    
    # mc_util.write_trip_tables(mc, dir_out + str(cc_fee) + '_2040_trips.omx')
        
    

✓ TAZ / land use / parking / zonal variables read. Time elapsed: 0.16 seconds
✓ skims read. Time elapsed: 19.33 seconds
✓ trip table read. Time elapsed: 29.93 seconds
✓ zonal variable tables generated. Time elapsed: 30.38 seconds
"Congestion Charge" enabled: $5 toll charge for auto trips entering central Boston.
Running for all purposes...
Mode choice for HBW started.
✓ Parameter table for HBW generated. Time elapsed: 30.87 seconds
✓ Trips for 0_PK calculated. Time elapsed: 50.88 seconds
✓ Trips for 1_PK calculated. Time elapsed: 65.21 seconds
✓ Trips for 0_OP calculated. Time elapsed: 80.85 seconds
✓ Trips for 1_OP calculated. Time elapsed: 95.94 seconds
Mode choice for HBO started.
✓ Parameter table for HBO generated. Time elapsed: 97.68 seconds
✓ Trips for 0_PK calculated. Time elapsed: 107.40 seconds
✓ Trips for 1_PK calculated. Time elapsed: 117.38 seconds
✓ Trips for 0_OP calculated. Time elapsed: 127.46 seconds
✓ Trips for 1_OP calculated. Time elapsed: 137.86 seconds
Mode choic

Mode choice for NHB started.
✓ Parameter table for NHB generated. Time elapsed: 139.51 seconds
✓ Trips for 0_PK calculated. Time elapsed: 151.38 seconds
✓ Trips for 1_PK calculated. Time elapsed: 163.09 seconds
✓ Trips for 0_OP calculated. Time elapsed: 174.22 seconds
✓ Trips for 1_OP calculated. Time elapsed: 185.60 seconds
Mode choice for HBSc1 started.
✓ Parameter table for HBSc1 generated. Time elapsed: 186.49 seconds
✓ Trips for 0_PK calculated. Time elapsed: 193.90 seconds
✓ Trips for 1_PK calculated. Time elapsed: 201.66 seconds
✓ Trips for 0_OP calculated. Time elapsed: 209.43 seconds
✓ Trips for 1_OP calculated. Time elapsed: 216.99 seconds
Mode choice for HBSc2 started.
✓ Parameter table for HBSc2 generated. Time elapsed: 217.74 seconds
✓ Trips for 0_PK calculated. Time elapsed: 225.02 seconds
✓ Trips for 1_PK calculated. Time elapsed: 235.00 seconds
✓ Trips for 0_OP calculated. Time elapsed: 243.72 seconds
✓ Trips for 1_OP calculated. Time elapsed: 251.69 seconds
Mode choice

Mode choice for HBSc3 started.
✓ Parameter table for HBSc3 generated. Time elapsed: 253.85 seconds
✓ Trips for 0_PK calculated. Time elapsed: 265.09 seconds
✓ Trips for 1_PK calculated. Time elapsed: 274.56 seconds
✓ Trips for 0_OP calculated. Time elapsed: 283.92 seconds
✓ Trips for 1_OP calculated. Time elapsed: 293.58 seconds
Scenario run is finished. You may now call methods in mc_util to produce output summaries.
✓ TAZ / land use / parking / zonal variables read. Time elapsed: 0.13 seconds
✓ skims read. Time elapsed: 18.79 seconds
✓ trip table read. Time elapsed: 27.90 seconds
✓ zonal variable tables generated. Time elapsed: 28.71 seconds
"Congestion Charge" enabled: $5 toll charge for auto trips entering central Boston.
Running for all purposes...
Mode choice for HBW started.
✓ Parameter table for HBW generated. Time elapsed: 28.85 seconds
✓ Trips for 0_PK calculated. Time elapsed: 51.36 seconds
✓ Trips for 1_PK calculated. Time elapsed: 66.71 seconds
✓ Trips for 0_OP calculated.